# Improve SS Performance - eliminate negative profit months

## Potential Problems From Visual Observations

* on winner and loosers there are buys and sells at levels of resistance that would loose
* there are multiple buys and sells on a flat line

## Improvements to try

1. if the next bars close is = to the last don't buy/sell (unsure this may reduce profit factor, the duplications are useful but maybe less so on the flat line) -- epic fail complete negative returns
2. near needs to be one bar away -- it already is < 2 is 1 bar away



3. there should be a buy if closes outside the bottom bb and bb pointing down
4. there should be a sell if closes outside the to bb and bb pointing up


5. reduce the average adr from 7 days to 3 days -- epic fail, complete negative returns

In [104]:
import numpy as np
import warnings
import pandas as pd
import pyperclip as clip
import ta
warnings.filterwarnings('ignore', category=FutureWarning)

128800 rows at 1m

102909 rows at 5min

In [105]:
csv_paths = ['data/BTCUSDT_1m_3m_back_from_2023-03-14.csv', 'data/BTCUSDT_1m_3m_back_from_2022-11-15.csv', 'data/BTCUSDT_1m_3m_back_from_2022-07-19.csv']
df = pd.DataFrame()
for csv_path in csv_paths:
    df = pd.concat([df, pd.read_csv(csv_path, usecols=['timestamp', 'open', 'high', 'low', 'close'])])

df['date'] = pd.to_datetime(df['timestamp']).dt.date 
df['timestamp'] = pd.to_datetime(df['timestamp'])
df.set_index('timestamp', inplace=True)
df.sort_index(inplace=True)
# df = df.resample('5T').agg({
#     'date': 'first',
#     'open': 'first',
#     'high': 'max',
#     'low': 'min',
#     'close': 'last'
# })
df.rename(columns={'close': 'Close', 'open': 'Open', 'high': 'High', 'low': 'Low'}, inplace=True)
df

,Open,High,Low,Close,date
timestamp,,,,,
2022-03-21 22:09:00,41336.2,41336.2,41284.6,41303.8,2022-03-21
2022-03-21 22:10:00,41303.8,41340.8,41287.4,41340.3,2022-03-21
2022-03-21 22:11:00,41340.2,41340.3,41317.9,41327.5,2022-03-21
2022-03-21 22:12:00,41327.5,41333.2,41282.8,41286.3,2022-03-21
2022-03-21 22:13:00,41286.2,41290.5,41273.0,41288.1,2022-03-21
...,...,...,...,...,...
2023-03-14 05:44:00,24306.7,24316.4,24290.0,24301.3,2023-03-14
2023-03-14 05:45:00,24301.3,24340.0,24296.7,24326.1,2023-03-14
2023-03-14 05:46:00,24326.1,24337.2,24309.8,24319.6,2023-03-14


In [106]:
def adr(df: pd.DataFrame) -> float:
    daily_high_low = df.groupby('date')['High', 'Low'].agg(['max', 'min'])
    daily_high_low['adr'] = daily_high_low[('High', 'max')] - daily_high_low[('Low', 'min')]
    return np.mean(daily_high_low['adr'])

def relative_adr_range_size(df_in: pd.DataFrame, resample_arg: str = 'W') -> str:
    # resample the dataframe into monthly periods
    groups = df_in.resample(resample_arg)
    df_out = pd.DataFrame()
    for _, group in groups:
        week_day_seg = group.copy()
        average_adr = adr(week_day_seg)
        week_day_seg['average_adr'] = average_adr
        df_out = pd.concat([df_out, week_day_seg])
    return df_out   

In [107]:
# alternative ? FIXME doesn't perform the same operation
# def relative_adr_range_size_alt(df: pd.DataFrame, resample_arg: str = 'W') -> pd.DataFrame:
#     df_out = df.groupby(pd.Grouper(freq=resample_arg)).apply(
#         lambda x: x.assign(average_adr=x['High'].diff().mean())
#     ).reset_index(drop=True)
#     return df_out

In [108]:
df2 = relative_adr_range_size(df.copy())
df2

,Open,High,Low,Close,date,average_adr
timestamp,,,,,,
2022-03-21 22:09:00,41336.2,41336.2,41284.6,41303.8,2022-03-21,815.40
2022-03-21 22:10:00,41303.8,41340.8,41287.4,41340.3,2022-03-21,815.40
2022-03-21 22:11:00,41340.2,41340.3,41317.9,41327.5,2022-03-21,815.40
2022-03-21 22:12:00,41327.5,41333.2,41282.8,41286.3,2022-03-21,815.40
2022-03-21 22:13:00,41286.2,41290.5,41273.0,41288.1,2022-03-21,815.40
...,...,...,...,...,...,...
2023-03-14 05:44:00,24306.7,24316.4,24290.0,24301.3,2023-03-14,587.05
2023-03-14 05:45:00,24301.3,24340.0,24296.7,24326.1,2023-03-14,587.05
2023-03-14 05:46:00,24326.1,24337.2,24309.8,24319.6,2023-03-14,587.05


In [109]:
# df3 = relative_adr_range_size_alt(df.copy())
# df3

In [110]:
def create_range_bar_df(df: pd.DataFrame):
        range_bars = []
        current_bar = {'average_adr': df.iloc[0]['average_adr'],'timestamp': df.index.to_series()[0], 'Open': df.iloc[0]['Open'], 'High': df.iloc[0]['High'], 'Low': df.iloc[0]['Low'], 'Close': df.iloc[0]['Close']}
        current_high = current_bar['High']
        current_low = current_bar['Low']

        for index, row in df.iterrows():
            high = row['High']
            low = row['Low']
            range_size = row['average_adr'] * 0.1

            if high - current_low >= range_size:
                current_bar['Close'] = current_low + range_size
                range_bars.append(current_bar)

                num_bars = int((high - current_low - range_size) // range_size)
                for i in range(num_bars):
                    current_bar = {'average_adr': row['average_adr'],'timestamp': index, 'Open': current_low + range_size * (i), 'High': current_low + range_size * (i + 1), 'Low': current_low + range_size * (i), 'Close': current_low + range_size * (i + 1)}
                    range_bars.append(current_bar)

                current_bar = {'average_adr': row['average_adr'],'timestamp': index, 'Open': current_low + range_size * num_bars, 'High': high, 'Low': current_low + range_size * num_bars, 'Close': row['Close']}
                current_high = high
                current_low = current_bar['Low']

            elif current_high - low >= range_size:
                current_bar['Close'] = current_high - range_size
                range_bars.append(current_bar)

                num_bars = int((current_high - low - range_size) // range_size)
                for i in range(num_bars):
                    current_bar = {'average_adr': row['average_adr'],'timestamp': index, 'Open': current_high - range_size * (i + 1), 'High': current_high - range_size * (i), 'Low': current_high - range_size * (i + 2), 'Close': current_high - range_size * (i + 1)}
                    range_bars.append(current_bar)

                current_bar = {'average_adr': row['average_adr'],'timestamp': index, 'Open': current_high - range_size * (num_bars + 1), 'High': current_high - range_size * num_bars, 'Low': low, 'Close': row['Close']}
                current_high = current_bar['High']
                current_low = low
            else:
                current_high = max(current_high, high)
                current_low = min(current_low, low)
                current_bar['timestamp'] = index
                current_bar['High'] = current_high
                current_bar['Low'] = current_low
                current_bar['Close'] = row['Close']
                current_bar['average_adr'] = row['average_adr']

        return pd.DataFrame(range_bars)

In [111]:
rb_df = create_range_bar_df(df2.copy())
rb_df['timestamp'] = pd.to_datetime(rb_df['timestamp'])
# can't use set_index because of duplicate timestamps
rb_df

,average_adr,timestamp,Open,High,Low,Close
0,815.40,2022-03-21 22:16:00,41336.20,41340.800,41266.7,41259.26
1,815.40,2022-03-21 22:17:00,41259.26,41340.800,41252.1,41259.26
2,815.40,2022-03-21 22:18:00,41259.26,41340.800,41250.0,41259.26
3,815.40,2022-03-21 22:19:00,41259.26,41340.800,41253.7,41259.26
4,815.40,2022-03-21 22:20:00,41259.26,41340.800,41241.0,41259.26
...,...,...,...,...,...,...
100511,587.05,2023-03-14 05:42:00,24317.79,24376.495,24290.0,24317.79
100512,587.05,2023-03-14 05:43:00,24317.79,24376.495,24306.8,24317.79
100513,587.05,2023-03-14 05:44:00,24317.79,24376.495,24290.0,24317.79
100514,587.05,2023-03-14 05:45:00,24317.79,24376.495,24296.7,24317.79


In [112]:

rb_df = rb_df.sort_values(by='timestamp')
rb_df


,average_adr,timestamp,Open,High,Low,Close
0,815.40,2022-03-21 22:16:00,41336.20,41340.800,41266.7,41259.26
1,815.40,2022-03-21 22:17:00,41259.26,41340.800,41252.1,41259.26
2,815.40,2022-03-21 22:18:00,41259.26,41340.800,41250.0,41259.26
3,815.40,2022-03-21 22:19:00,41259.26,41340.800,41253.7,41259.26
4,815.40,2022-03-21 22:20:00,41259.26,41340.800,41241.0,41259.26
...,...,...,...,...,...,...
100511,587.05,2023-03-14 05:42:00,24317.79,24376.495,24290.0,24317.79
100512,587.05,2023-03-14 05:43:00,24317.79,24376.495,24306.8,24317.79
100513,587.05,2023-03-14 05:44:00,24317.79,24376.495,24290.0,24317.79
100514,587.05,2023-03-14 05:45:00,24317.79,24376.495,24296.7,24317.79


In [113]:
def create_range_bar_df2(df: pd.DataFrame, range_size_percent: float = 0.1):
        # Calculate range size as 10% of average daily range (ADR)
        range_size = df['average_adr'] * range_size_percent
        # Determine when a new range bar starts based on range size
        starts_new_bar = ((df['High'].shift() - df['Low']) >= range_size) | ((df['High'] - df['Low'].shift()) >= range_size)
        # Assign each row to a range bar using cumulative sum of starts_new_bar
        range_bar_id = starts_new_bar.cumsum()
        # Calculate open, high, low, close for each range bar
        range_bars = df.groupby(range_bar_id).apply(
            lambda x: {
                'timestamp': x.index[0],
                'average_adr': x['average_adr'].iloc[0],
                'Open': x['Low'].iloc[0],
                'High': x['High'].max(),
                'Low': x['Low'].min(),
                'Close': x['Close'].iloc[-1]
            }
        ).to_list()

        return pd.DataFrame(range_bars)

In [114]:
# rb_df2 = create_range_bar_df2(df2.copy())
# rb_df2

add macd, we select 26 for the slow moving average, 12 for the fast moving average and 9 for the signal line.

In [115]:
# calculate MACD with custom parameters
macd = ta.trend.MACD(rb_df['Close'], window_slow=26, window_fast=12, window_sign=9)
# add MACD values to the dataframe as new columns
rb_df['macd'] = macd.macd()
rb_df['macd_signal'] = macd.macd_signal()
rb_df['macd_histogram'] = macd.macd_diff()
rb_df.dropna(inplace=True)
rb_df


,average_adr,timestamp,Open,High,Low,Close,macd,macd_signal,macd_histogram
33,815.40,2022-03-21 22:49:00,41177.72,41259.260,41160.0,41177.72,-13.756046,-16.656924,2.900877
34,815.40,2022-03-21 22:50:00,41177.72,41259.260,41169.4,41177.72,-12.931911,-15.911921,2.980010
35,815.40,2022-03-21 22:51:00,41177.72,41259.260,41172.7,41177.72,-12.138848,-15.157307,3.018458
36,815.40,2022-03-21 22:52:00,41177.72,41259.260,41132.0,41177.72,-11.379169,-14.401679,3.022510
37,815.40,2022-03-21 22:53:00,41177.72,41259.260,41130.0,41177.72,-10.654301,-13.652203,2.997902
...,...,...,...,...,...,...,...,...,...
100511,587.05,2023-03-14 05:42:00,24317.79,24376.495,24290.0,24317.79,-29.846977,-33.653847,3.806871
100512,587.05,2023-03-14 05:43:00,24317.79,24376.495,24306.8,24317.79,-28.753354,-32.673748,3.920394
100513,587.05,2023-03-14 05:44:00,24317.79,24376.495,24290.0,24317.79,-27.568854,-31.652769,4.083916
100514,587.05,2023-03-14 05:45:00,24317.79,24376.495,24296.7,24317.79,-26.326651,-30.587546,4.260895


add bb we select 12 for the moving average and 2 for the standard deviation.

In [116]:
bb = ta.volatility.BollingerBands(rb_df['Close'], window=12, window_dev=2)

# add upper and lower Bollinger Bands to the dataframe as new columns
rb_df['bb_upper'] = bb.bollinger_hband()
rb_df['bb_lower'] = bb.bollinger_lband()
rb_df.dropna(inplace=True)
rb_df


,average_adr,timestamp,Open,High,Low,Close,macd,macd_signal,macd_histogram,bb_upper,bb_lower
44,815.40,2022-03-21 22:59:00,41096.18,41177.720,41090.1,41096.18,-13.063006,-10.245583,-2.817424,41215.997931,41125.852069
45,815.40,2022-03-21 23:00:00,41096.18,41177.720,41066.9,41096.18,-17.630299,-11.722526,-5.907773,41224.906328,41103.353672
46,815.40,2022-03-21 23:01:00,41096.18,41177.720,41066.7,41096.18,-21.007746,-13.579570,-7.428177,41227.950711,41086.719289
47,815.40,2022-03-21 23:02:00,41096.18,41177.720,41067.4,41096.18,-23.414490,-15.546554,-7.867936,41227.416649,41073.663351
48,815.40,2022-03-21 23:04:00,41096.18,41177.720,41088.9,41096.18,-25.033284,-17.443900,-7.589384,41224.144524,41063.345476
...,...,...,...,...,...,...,...,...,...,...,...
100511,587.05,2023-03-14 05:42:00,24317.79,24376.495,24290.0,24317.79,-29.846977,-33.653847,3.806871,24405.041954,24279.548046
100512,587.05,2023-03-14 05:43:00,24317.79,24376.495,24306.8,24317.79,-28.753354,-32.673748,3.920394,24385.560448,24284.308718
100513,587.05,2023-03-14 05:44:00,24317.79,24376.495,24290.0,24317.79,-27.568854,-31.652769,4.083916,24383.306271,24281.626229
100514,587.05,2023-03-14 05:45:00,24317.79,24376.495,24296.7,24317.79,-26.326651,-30.587546,4.260895,24371.330290,24283.818043


add rsi we use a value of 7 for the RSI.

In [117]:
rsi = ta.momentum.RSIIndicator(rb_df['Close'], window=7).rsi()
rb_df['rsi'] = rsi
rb_df.dropna(inplace=True)
rb_df.head(20)


,average_adr,timestamp,Open,High,Low,Close,macd,macd_signal,macd_histogram,bb_upper,bb_lower,rsi
50,815.4,2022-03-21 23:10:00,41096.18,41177.72,41090.10,41096.18,-26.489970,-20.624697,-5.865274,41210.554524,41049.755476,100.0
51,815.4,2022-03-21 23:12:00,41096.18,41177.72,41070.10,41096.18,-26.559207,-21.811599,-4.747608,41200.236649,41046.483351,100.0
52,815.4,2022-03-21 23:14:00,41096.18,41177.72,41075.60,41096.18,-26.310783,-22.711435,-3.599347,41187.180711,41045.949289,100.0
53,815.4,2022-03-21 23:15:00,41096.18,41177.72,41087.60,41096.18,-25.816311,-23.332410,-2.483900,41170.546328,41048.993672,100.0
54,815.4,2022-03-21 23:16:00,41096.18,41177.72,41087.90,41096.18,-25.134701,-23.692869,-1.441832,41148.047931,41057.902069,100.0
55,815.4,2022-03-21 23:17:00,41096.18,41177.72,41071.60,41096.18,-24.314241,-23.817143,-0.497098,41096.180060,41096.179940,100.0
56,815.4,2022-03-21 23:18:00,41096.18,41177.72,41077.20,41096.18,-23.394345,-23.732583,0.338239,41096.180060,41096.179940,100.0
57,815.4,2022-03-21 23:19:00,41096.18,41177.72,41014.64,41096.18,-22.407026,-23.467472,1.060446,41096.180060,41096.179940,100.0
58,815.4,2022-03-21 23:23:00,41014.64,41096.18,41008.10,41014.64,-27.882751,-24.350528,-3.532223,41134.457931,41044.312069,0.0
59,815.4,2022-03-21 23:24:00,41014.64,41096.18,40960.60,41014.64,-31.855096,-25.851441,-6.003655,41143.366328,41021.813672,0.0


In [118]:
rb_df.reset_index(drop=True, inplace=True)
rb_df['signal'] = 0
rb_df['false_signal'] = -1
rb_df

,average_adr,timestamp,Open,High,Low,Close,macd,macd_signal,macd_histogram,bb_upper,bb_lower,rsi,signal,false_signal
0,815.40,2022-03-21 23:10:00,41096.18,41177.720,41090.1,41096.18,-26.489970,-20.624697,-5.865274,41210.554524,41049.755476,100.000000,0,-1
1,815.40,2022-03-21 23:12:00,41096.18,41177.720,41070.1,41096.18,-26.559207,-21.811599,-4.747608,41200.236649,41046.483351,100.000000,0,-1
2,815.40,2022-03-21 23:14:00,41096.18,41177.720,41075.6,41096.18,-26.310783,-22.711435,-3.599347,41187.180711,41045.949289,100.000000,0,-1
3,815.40,2022-03-21 23:15:00,41096.18,41177.720,41087.6,41096.18,-25.816311,-23.332410,-2.483900,41170.546328,41048.993672,100.000000,0,-1
4,815.40,2022-03-21 23:16:00,41096.18,41177.720,41087.9,41096.18,-25.134701,-23.692869,-1.441832,41148.047931,41057.902069,100.000000,0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100461,587.05,2023-03-14 05:42:00,24317.79,24376.495,24290.0,24317.79,-29.846977,-33.653847,3.806871,24405.041954,24279.548046,37.974584,0,-1
100462,587.05,2023-03-14 05:43:00,24317.79,24376.495,24306.8,24317.79,-28.753354,-32.673748,3.920394,24385.560448,24284.308718,37.974584,0,-1
100463,587.05,2023-03-14 05:44:00,24317.79,24376.495,24290.0,24317.79,-27.568854,-31.652769,4.083916,24383.306271,24281.626229,37.974584,0,-1
100464,587.05,2023-03-14 05:45:00,24317.79,24376.495,24296.7,24317.79,-26.326651,-30.587546,4.260895,24371.330290,24283.818043,37.974584,0,-1


In [119]:
def iterations_back_till_condition(series, condition):
    count = 0
    for value in series[::-1]:
        print(f'value: {value}')
        if condition(value):
            print(f'Condition met at index {value}')
            break
        count += 1
    return count

# create example series
s = pd.Series([1, 2, 3, 4, 5])

# pass the equality operator as a lambda function
count = iterations_back_till_condition(s, lambda x: x == 2)

# print the number of iterations before the condition was true
print(f'Number of iterations before condition was true: {count}')

value: 5
value: 4
value: 3
value: 2
Condition met at index 2
Number of iterations before condition was true: 3


In [120]:
def mark_false_signals(df_sample: pd.DataFrame, indexes: list, expect_long: bool):
    from scipy.stats import linregress
    # count false signals
    false_signals = 0
    true_signals = 0
    for idx in indexes:
        try:
            seg = list(df_sample[idx:idx+3]['Close'])
            # print(f'long seg: {str(seg)}')
            slope, _, _, _, _ = linregress(range(len(seg)), seg)
            if expect_long:
                if slope < 0:
                    false_signals += 1
                    df_sample.loc[idx, 'false_signal'] = 1
                else:
                    df_sample.loc[idx, 'false_signal'] = 0
                    true_signals += 1
            else:
                if slope > 0:
                    false_signals += 1
                    df_sample.loc[idx, 'false_signal'] = 1
                else:
                    df_sample.loc[idx, 'false_signal'] = 0
                    true_signals += 1            
        except Exception as e:
            print(f'exception: {str(e)}')       
    print(f'false_signals: {false_signals}, true_signals: {true_signals}, diff: {true_signals - false_signals}')     

In [121]:
class RangeBarStrategy:
    
    long_indexes = []
    short_indexes = []
  
    def __init__(self, df):
        self.df = df
        self.updated_df = pd.DataFrame()

    def bb_upper_near(self, index, row):
        try:
            upper_series = self.df.iloc[:index+1]['bb_upper']
            if len(upper_series) < 2:
                return False
            # print(f'upper_series:\n{str(upper_series)}')
            close = row['Close']
            since = self.iterations_back_till_condition(upper_series, lambda x: x >= close)
            # print(f'bb_upper_near(upper_series >= close): index: {index}, close: {close}, since: {since}')
            return since < 2
        except Exception as e:
            print(f'bb_upper_near: exception: {e.__cause__}')
            raise e
        

    def bb_lower_near(self, index, row):
       try:
         lower_series = self.df.iloc[:index+1]['bb_lower']
         if len(lower_series) < 2:
                return False
        #  print(f'lower_series:\n{str(lower_series)}')
         close = row['Close']
         since = self.iterations_back_till_condition(lower_series, lambda x: x <= close)
        #  print(f'bb_lower_near(lower_series <= close): index: {index}, close: {close}, since: {since}')
         return since < 2
       except Exception as e:
            print(f'bb_lower_near: exception: {e.__cause__}')
            raise e
       
    def iterations_back_till_condition(self, series, condition):
        count = 0
        for value in series[::-1]:
            if condition(value):
                break
            count += 1
        return count
       
    def bb_upper_pointing_up(self, index):
        from scipy.stats import linregress
        bb_seg = self.df.iloc[index-3:index+1]['bb_upper']
        # print(f'bb_seg: {len(bb_seg)}')
        if len(bb_seg) > 0:
            seg_len = len(bb_seg)
            try:
                slope, _, _, _, _ = linregress(range(seg_len), bb_seg)
                # print(f'bb_upper_pointing_up: seg_len: {seg_len}, slope: {slope}')
                return slope > 0
            except Exception as e:
                print(f'bb_upper_pointing_up: exception: {str(e)}')
        return False

    def bb_lower_pointing_down(self, index):
        from scipy.stats import linregress
        bb_seg = self.df.iloc[index-3:index+1]['bb_lower']
        # print(f'bb_seg: {len(bb_seg)}')
        if len(bb_seg) > 0:
            seg_len = len(bb_seg)
            try:
                slope, _, _, _, _ = linregress(range(seg_len), bb_seg)
                # print(f'bb_lower_pointing_down: seg_len: {seg_len}, slope: {slope}')
                return slope < 0
            except Exception as e:
                print(f'bb_lower_pointing_down: exception: {str(e)}')  
           
        return False
    
    def scan(self):
        for index, row in self.df.iterrows():
            try:
                updated_row = self.next(row, index)
                self.df.loc[index] = updated_row
            except Exception as e:
                print(f'next: exception: {str(e)}')

    def next(self, row, index):
        is_long_rsi = row['rsi'] > 70
        is_long_macd = row['macd'] > row['macd_signal'] > 0
        is_bb_upper_near = self.bb_upper_near(index, row)
        is_bb_upper_pointing_up = self.bb_upper_pointing_up(index)

        is_short_rsi = row['rsi'] < 30
        is_short_macd = row['macd'] < row['macd_signal'] < 0
        is_bb_lower_near = self.bb_lower_near(index, row)
        is_bb_lower_pointing_down = self.bb_lower_pointing_down(index)

        # is_not_eq_last_close = row['Close'] != self.df.iloc[index-1]['Close']

        if is_long_rsi and is_long_macd and is_bb_upper_near and is_bb_upper_pointing_up:
            self.long_indexes.append(index)
            row['signal'] = 1
        elif is_short_rsi and is_short_macd and is_bb_lower_near and is_bb_lower_pointing_down:
            row['signal'] = -1
            self.short_indexes.append(index)
        else:
            row['signal'] = 0
        return row       

    def print(self):
        print(f'long_indexes: {self.long_indexes}')
        print(f'short_indexes: {self.short_indexes}')


In [122]:
df_sample = rb_df
strategy = RangeBarStrategy(df_sample)
strategy.scan()
print(f'long_indexes: {len(strategy.long_indexes)}')
print(f'short_indexes: {len(strategy.short_indexes)}')

long_indexes: 9501
short_indexes: 9930


## With is_not_eq_last_close
### 3 day average adr
* long_indexes: 1360
* short_indexes: 1600
### 7 day average adr
* long_indexes: 1872
* short_indexes: 2083

### using 3 on near
false_signals: 1697, true_signals: 8707, diff: 7010
false_signals: 1841, true_signals: 9182, diff: 7341
### using 2 on near
false_signals: 1549, true_signals: 7952, diff: 6403
false_signals: 1664, true_signals: 8266, diff: 6602

In [123]:
mark_false_signals(df_sample, strategy.long_indexes, True)
mark_false_signals(df_sample, strategy.short_indexes, False)
df_sample

false_signals: 1549, true_signals: 7952, diff: 6403
false_signals: 1664, true_signals: 8266, diff: 6602


,average_adr,timestamp,Open,High,Low,Close,macd,macd_signal,macd_histogram,bb_upper,bb_lower,rsi,signal,false_signal
0,815.40,2022-03-21 23:10:00,41096.18,41177.720,41090.1,41096.18,-26.489970,-20.624697,-5.865274,41210.554524,41049.755476,100.000000,0,-1
1,815.40,2022-03-21 23:12:00,41096.18,41177.720,41070.1,41096.18,-26.559207,-21.811599,-4.747608,41200.236649,41046.483351,100.000000,0,-1
2,815.40,2022-03-21 23:14:00,41096.18,41177.720,41075.6,41096.18,-26.310783,-22.711435,-3.599347,41187.180711,41045.949289,100.000000,0,-1
3,815.40,2022-03-21 23:15:00,41096.18,41177.720,41087.6,41096.18,-25.816311,-23.332410,-2.483900,41170.546328,41048.993672,100.000000,0,-1
4,815.40,2022-03-21 23:16:00,41096.18,41177.720,41087.9,41096.18,-25.134701,-23.692869,-1.441832,41148.047931,41057.902069,100.000000,0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100461,587.05,2023-03-14 05:42:00,24317.79,24376.495,24290.0,24317.79,-29.846977,-33.653847,3.806871,24405.041954,24279.548046,37.974584,0,-1
100462,587.05,2023-03-14 05:43:00,24317.79,24376.495,24306.8,24317.79,-28.753354,-32.673748,3.920394,24385.560448,24284.308718,37.974584,0,-1
100463,587.05,2023-03-14 05:44:00,24317.79,24376.495,24290.0,24317.79,-27.568854,-31.652769,4.083916,24383.306271,24281.626229,37.974584,0,-1
100464,587.05,2023-03-14 05:45:00,24317.79,24376.495,24296.7,24317.79,-26.326651,-30.587546,4.260895,24371.330290,24283.818043,37.974584,0,-1


In [ ]:
df_sample.columns

## Plot shows false signals in different color - for purpose of figuring out how to eliminate them

In [ ]:
from bokeh.plotting import figure, show, output_file
from bokeh.models import ColumnDataSource, CDSView, BooleanFilter

# Create a ColumnDataSource object to hold the plot data
source = ColumnDataSource(data=dict(
    x=df_sample.index,
    y=df_sample['Close'],
    bb_upper=df_sample['bb_upper'],
    bb_lower=df_sample['bb_lower'],
    signal=df_sample['signal'],
    false_signal=df_sample['false_signal']
))

# Create a view of the source using a BooleanFilter
false_buy_filter = BooleanFilter([True if s==1 and fs==1 else False for s,fs in zip(source.data['signal'], source.data['false_signal'])])
false_sell_filter = BooleanFilter([True if s==-1 and fs==1 else False for s,fs in zip(source.data['signal'], source.data['false_signal'])])
false_buy_view = CDSView(source=source, filters=[false_buy_filter])
false_sell_view = CDSView(source=source, filters=[false_sell_filter])
buy_filter = BooleanFilter([True if s==1 and fs==0 else False for s,fs in zip(source.data['signal'], source.data['false_signal'])])
sell_filter = BooleanFilter([True if s==-1 and fs==0 else False for s,fs in zip(source.data['signal'], source.data['false_signal'])])
buy_view = CDSView(source=source, filters=[buy_filter])
sell_view = CDSView(source=source, filters=[sell_filter])

# Create a new figure object
p = figure(title='ss', x_axis_label='Date',
           y_axis_label='Price', width=1400)

# Plot the Close values as a line
p.line(x='x', y='y', source=source, legend_label='Close')

# Plot the bb_upper and bb_lower values as shaded areas
p.varea(x='x', y1='bb_upper', y2='bb_lower', source=source, alpha=0.2, legend_label='Bollinger Bands')

# Mark the long signals with green triangles and the short signals with red triangles
p.triangle(x='x', y='y', source=source, view=buy_view, color='green', size=10, legend_label='Buy Signal')
p.inverted_triangle(x='x', y='y', source=source, view=sell_view, color='red', size=10, legend_label='Sell Signal')

p.triangle(x='x', y='y', source=source, view=false_buy_view, color='black', size=10, legend_label='False Buy Signal')
p.inverted_triangle(x='x', y='y', source=source, view=false_sell_view, color='black', size=10, legend_label='False Sell Signal')

# Show the legend
# p.legend.location = "bottom_right"

# Show the plot
show(p)
output_file('charts/ss_strategy_1min_7d_avg_adr_range_bars_improvements.html')


In [ ]:
df_sample.to_csv('tmp-data/ss_strategy_1min_7d_avg_adr_range_bars.csv')